In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import nibabel as nib
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torchsummary
from torchvision import transforms
import time
from sklearn.metrics import classification_report

In [4]:
match_folder=np.loadtxt('/home/sumins/workspace/washout classification/base_data/condition_match_folder.txt',dtype=str)
match_folder=match_folder.tolist()

In [5]:
import ast
range_path='/home/sumins/workspace/washout classification/base_data/liverrange.txt'
range_value_path='/home/sumins/workspace/washout classification/base_data/liverrange_value.txt'

liverrange=[]
liverrange_value=[]
with open (range_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange.append(ast.literal_eval(line))

with open (range_value_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange_value.append(ast.literal_eval(line))

In [6]:
same=0
diff=0
diff_index=[]
for index, i in enumerate(liverrange_value):
    if i[0]==i[1]==i[2]==i[3]:
        same+=1
    else:
        diff+=1
        diff_index.append(index)
        

In [7]:
print("same ",same)
print("diff ", diff)

same  146
diff  19


In [8]:
for k,i in enumerate(diff_index):
    del match_folder[i-k]
    del liverrange[i-k]
    del liverrange_value[i-k]

In [9]:
Tumor_slice=[]
data_path='/disk1/HCC2'
for i in match_folder:    
    washout_dir = os.path.join(data_path, 'Anonymize_'+i,'washout_'+i+'.txt') 
    washout=[]
    with open(washout_dir,'r') as f:
        lines=f.readlines()
        slice=[]
        label=[]
        for line in lines:
            line=line.strip()
            washout.append(line)
        
        for i in [washout[0],washout[-1]]:
            k=i.split('\t')
            slice.append(int(k[0]))

    Tumor_slice.append(slice) 


In [10]:
Tumor_slice[1]

[80, 87]

In [11]:
import ast

path1='/home/sumins/workspace/washout classification/min_slice/arterial.txt'
path2='/home/sumins/workspace/washout classification/min_slice/pre.txt'
path3='/home/sumins/workspace/washout classification/min_slice/delay.txt'
path4='/home/sumins/workspace/washout classification/min_slice/portal.txt'

arterial=[]
pre=[]
delay=[]
portal=[]

with open (path1,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        arterial.append(ast.literal_eval(line))
with open (path2,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        pre.append(ast.literal_eval(line))
with open (path3,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        delay.append(ast.literal_eval(line))
with open (path4,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        portal.append(ast.literal_eval(line))

In [12]:
for k,i in enumerate(diff_index):
    del arterial[i-k]
    del pre[i-k]
    del portal[i-k]
    del delay[i-k]

In [13]:
#for i in range(len(Tumor_slice)):
liver_label=[]

for j in range(len(liverrange)):
    liver_one=[]
    for i in range(liverrange[j][4],liverrange[j][5]+1):
        if i in range(Tumor_slice[j][0],Tumor_slice[j][1]+1):
            liver_one.append(1)
        else:
            liver_one.append(0)
    liver_label.append(liver_one)
        

In [14]:
Train_slice=liver_label[:120]
Test_slice=liver_label[120:]

In [15]:
Tumor=0
clear=0
for i in Train_slice:
    for j in i:
        if j==1:
            Tumor+=1
        else:
            clear+=1
print("Tumor: {}, Clear: {}".format(Tumor, clear))
            

Tumor: 1364, Clear: 3725


In [16]:
Tumor=0
clear=0
for i in Test_slice:
    for j in i:
        if j==1:
            Tumor+=1
        else:
            clear+=1
print("Tumor: {}, Clear: {}".format(Tumor, clear))

Tumor: 294, Clear: 783


In [17]:
len(liver_label)

146

In [18]:
liverrange_value[2]

[52, 52, 52, 52]

In [20]:
A_DATA=[]
R_DATA=[]
P_DATA=[]   
D_DATA=[]
all_phase=[]
LABEL=[]
data_path='/disk1/HCC2'
for i in range(len(Train_slice)):
    folder_name=match_folder[i]
    a_path=os.path.join(data_path, 'Anonymize_'+folder_name,'A_Anonymize_'+folder_name+'.nii') 
    a_data_all=nib.load(a_path).get_fdata()
    r_path=os.path.join(data_path, 'Anonymize_'+folder_name,'R_Anonymize_'+folder_name+'.nii') 
    r_data_all=nib.load(r_path).get_fdata()
    p_path=os.path.join(data_path, 'Anonymize_'+folder_name,'P_Anonymize_'+folder_name+'.nii') 
    p_data_all=nib.load(p_path).get_fdata()
    d_path=os.path.join(data_path, 'Anonymize_'+folder_name,'D_Anonymize_'+folder_name+'.nii') 
    d_data_all=nib.load(d_path).get_fdata()

    for j in range(0,liverrange_value[i][0]):
        r=r_data_all[:,:,pre[i][j]-1].astype(dtype='float32')
        a=a_data_all[:,:,arterial[i][j]-1].astype(dtype='float32')
        p=p_data_all[:,:,portal[i][j]-1].astype(dtype='float32')
        d=d_data_all[:,:,delay[i][j]-1].astype(dtype='float32')
        DATASET=np.stack([r,a,p,d],axis=-1)
        all_phase.append(DATASET)

        LABEL.append(Train_slice[i][j])
    a_data_all=0
    r_data_all=0
    p_data_all=0
    d_data_all=0
    print(i, "done")


        

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done


In [21]:
print(len(all_phase))

5089


In [22]:
k=np.array(all_phase)

data=nib.Nifti1Image(k,None)

In [23]:
nib.save(data,'/home/sumins/workspace/washout classification/transfer_learning/channel4_train.nii')

In [24]:
np.savetxt("/home/sumins/workspace/washout classification/transfer_learning/label_train.txt",LABEL,fmt='%d')

In [ ]:
A_DATA=[]
R_DATA=[]
P_DATA=[]   
D_DATA=[]
all_phase=[]
LABEL=[]
data_path='/disk1/HCC2'
for i in range(len(Test_slice)):
    folder_name=match_folder[i]
    a_path=os.path.join(data_path, 'Anonymize_'+folder_name,'A_Anonymize_'+folder_name+'.nii') 
    a_data_all=nib.load(a_path).get_fdata()
    r_path=os.path.join(data_path, 'Anonymize_'+folder_name,'R_Anonymize_'+folder_name+'.nii') 
    r_data_all=nib.load(r_path).get_fdata()
    p_path=os.path.join(data_path, 'Anonymize_'+folder_name,'P_Anonymize_'+folder_name+'.nii') 
    p_data_all=nib.load(p_path).get_fdata()
    d_path=os.path.join(data_path, 'Anonymize_'+folder_name,'D_Anonymize_'+folder_name+'.nii') 
    d_data_all=nib.load(d_path).get_fdata()

    for j in range(0,liverrange_value[i][0]):
        r=r_data_all[:,:,pre[i][j]-1].astype(dtype='float32')
        a=a_data_all[:,:,arterial[i][j]-1].astype(dtype='float32')
        p=p_data_all[:,:,portal[i][j]-1].astype(dtype='float32')
        d=d_data_all[:,:,delay[i][j]-1].astype(dtype='float32')
        DATASET=np.stack([r,a,p,d],axis=-1)
        all_phase.append(DATASET)

        LABEL.append(Train_slice[i][j])
    a_data_all=0
    r_data_all=0
    p_data_all=0
    d_data_all=0
    print(i, "done")


        